## 라이브러리 import

In [1]:
import time # 걸리는 시간 확인
import pandas as pd # 데이터 분석
import numpy as np # 수치 계산
from glob import glob # 파일 경로 확인
from dask import dataframe as dd # dask로 대용량 파일 로드
from dask.diagnostics import ProgressBar # dask progress bar

pbar = ProgressBar()
pbar.register()

## 파일 경로 변수

In [2]:
file_name = glob('data/*.csv')
file_name

['data/2019-Nov.csv', 'data/2019-Oct.csv']

## 파일 로드(dask)

### 2019-Oct.csv (6GB)

In [3]:
# ddf : dask dataframe
# dd.read_csv() : dask 로 csv 로드하기
ddf = dd.read_csv(file_name[1], dtype={'event_type':'category', 'product_id':'uint32', 'category_id':'uint64', 'category_code':'category', 'brand':'category', 'price':'float32', 'user_id':'uint32'})
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, event_time to user_session
dtypes: category(3), object(2), float32(1), uint32(2), uint64(1)

In [4]:
# dask는 가상 데이터 프레임을 먼저 보여줌
# 데이터 값이 보이지 않는 것은 아직 메모리 상에 올려놓지 않았기 때문
ddf

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
npartitions=88,,,,,,,,,
,object,category[unknown],uint32,uint64,category[unknown],category[unknown],float32,uint32,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [5]:
# head를 해주면 출력해줌
ddf.head()

[########################################] | 100% Completed | 424.64 ms


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [6]:
# pandas처럼 연산도 가능하지만 연산 함수만 적용하면 결과물은 출력되지 않음
ddf['event_type'].nunique()

dd.Scalar<series-..., dtype=int64>

In [7]:
# compute()를 해주어야 결과물이 출력됨
# 다만 연산결과물을 로드하기까지 시간이 다소 걸림
ddf['event_type'].nunique().compute()

[########################################] | 100% Completed | 15.65 s


3

## 중간 결론
- DASK는 대용량 데이터를 노트북 상에 로드하는 시간에 이점이 있다.
    - 병렬 컴퓨팅(?)과 같은 방식을 사용하기 때문인 듯
- DASK는 데이터프레임의 연산 결과물의 출력에는 시간이 다소 오래 걸린다.
- DASK로 우선 대용량 csv를 로드한 후, compute()를 통해 pandas 데이터프레임 형식으로 변환하여 EDA를 진행하는 것이 빠를 수도 있을 것 같다.
- DASK로도 충분히 파생변수 생성이나 group연산이 가능하지만 pandas와 문법이 다른 부분이 익숙치 않기 때문에 로드 이후에는 pandas로 진행하는 것이 편할 듯

참고 : https://devtimes.com/python-dask/

In [8]:
# dask로 로드한 데이터프레임을 compute()를 통해 pandas 데이터프레임 형태로 변환하기
# time을 활용한 이유는 dask progress bar의 속도와 실제 속도에 다소 차이가 있는 것 같아 비교하기 위함
start = time.time()
df = ddf.compute()
df
end = time.time()

print('걸린 시간 : ', (end - start), 'sec')

[########################################] | 100% Completed | 14.91 s
걸린 시간 :  18.50674605369568 sec


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 483207
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     uint32  
 3   category_id    uint64  
 4   category_code  category
 5   brand          category
 6   price          float32 
 7   user_id        uint32  
 8   user_session   object  
dtypes: category(3), float32(1), object(2), uint32(2), uint64(1)
memory usage: 1.9+ GB


In [10]:
df.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc


In [11]:
# dask 데이터프레임에서 진행한 함수를 실행한 결과 pandas가 훨씬 빠름
df['event_type'].nunique()

3

### 2019-Nov.csv (9GB)

In [12]:
ddf2 = dd.read_csv(file_name[0], dtype={'event_type':'category', 'product_id':'uint32', 'category_id':'uint64', 'category_code':'category', 'brand':'category', 'price':'float32', 'user_id':'uint32'})
ddf2.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, event_time to user_session
dtypes: category(3), object(2), float32(1), uint32(2), uint64(1)

In [13]:
start = time.time()
df2 = ddf2.compute()
df2
end = time.time()

print('걸린 시간 : ', (end - start), 'sec')

[########################################] | 100% Completed | 23.66 s
걸린 시간 :  31.628165245056152 sec


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67501979 entries, 0 to 482472
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     uint32  
 3   category_id    uint64  
 4   category_code  category
 5   brand          category
 6   price          float32 
 7   user_id        uint32  
 8   user_session   object  
dtypes: category(3), float32(1), object(2), uint32(2), uint64(1)
memory usage: 3.1+ GB


In [15]:
df2.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.070007,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.649994,530496790,8e5f4f83-366c-4f70-860e-ca7417414283


## 전체 데이터프레임 확인(pandas)

In [16]:
# df : 2019-Oct.csv
# df2 : 2019-Nov.csv
df.shape, df2.shape

((42448764, 9), (67501979, 9))

In [17]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 483207
Data columns (total 9 columns):
 #   Column         Non-Null Count     Dtype   
---  ------         --------------     -----   
 0   event_time     42448764 non-null  object  
 1   event_type     42448764 non-null  category
 2   product_id     42448764 non-null  uint32  
 3   category_id    42448764 non-null  uint64  
 4   category_code  28933155 non-null  category
 5   brand          36331684 non-null  category
 6   price          42448764 non-null  float32 
 7   user_id        42448764 non-null  uint32  
 8   user_session   42448762 non-null  object  
dtypes: category(3), float32(1), object(2), uint32(2), uint64(1)
memory usage: 1.9+ GB


In [18]:
df2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67501979 entries, 0 to 482472
Data columns (total 9 columns):
 #   Column         Non-Null Count     Dtype   
---  ------         --------------     -----   
 0   event_time     67501979 non-null  object  
 1   event_type     67501979 non-null  category
 2   product_id     67501979 non-null  uint32  
 3   category_id    67501979 non-null  uint64  
 4   category_code  45603808 non-null  category
 5   brand          58277901 non-null  category
 6   price          67501979 non-null  float32 
 7   user_id        67501979 non-null  uint32  
 8   user_session   67501969 non-null  object  
dtypes: category(3), float32(1), object(2), uint32(2), uint64(1)
memory usage: 3.1+ GB


In [19]:
display(df.describe(), df2.describe())

,product_id,category_id,price,user_id
count,4.244876e+07,4.244876e+07,4.244876e+07,4.244876e+07
mean,1.054993e+07,2.057404e+18,2.903238e+02,5.335371e+08
std,1.188191e+07,1.843926e+16,3.363253e+02,1.852374e+07
min,1.000978e+06,2.053014e+18,0.000000e+00,3.386938e+07
25%,1.005157e+06,2.053014e+18,6.598000e+01,5.159043e+08
50%,5.000470e+06,2.053014e+18,1.629300e+02,5.296965e+08
75%,1.600030e+07,2.053014e+18,3.585700e+02,5.515788e+08
max,6.050001e+07,2.175420e+18,2.574070e+03,5.662809e+08


,product_id,category_id,price,user_id
count,6.750198e+07,6.750198e+07,6.750198e+07,6.750198e+07
mean,1.251406e+07,2.057898e+18,2.924591e+02,5.386397e+08
std,1.725741e+07,2.012549e+16,3.373027e+02,2.288516e+07
min,1.000365e+06,2.053014e+18,0.000000e+00,1.030022e+07
25%,1.305977e+06,2.053014e+18,6.924000e+01,5.164762e+08
50%,5.100568e+06,2.053014e+18,1.657700e+02,5.350573e+08
75%,1.730075e+07,2.053014e+18,3.603400e+02,5.610794e+08
max,1.000286e+08,2.187708e+18,2.574070e+03,5.799699e+08


In [20]:
display(df.describe(include='O'), df2.describe(include='O'))

,event_time,user_session
count,42448764,42448762
unique,2621538,9244421
top,2019-10-15 08:52:00 UTC,fb075266-182d-4c11-b5f7-4e4dcdabd4a7
freq,116,1159


,event_time,user_session
count,67501979,67501969
unique,2549559,13776050
top,2019-11-15 02:51:51 UTC,d99d91bf-40f8-4e29-9593-54b4a1826542
freq,243,4128


## 데이터프레임 합치기

In [21]:
df.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [22]:
len(df) + len(df2)

109950743

In [23]:
df_all = pd.concat([df, df2], ignore_index=True)
df_all

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
109950738,2019-11-30 23:59:58 UTC,view,15700137,2053013559733912211,NaN,NaN,277.739990,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b
109950739,2019-11-30 23:59:58 UTC,view,28719425,2053013565639492569,apparel.shoes,baden,62.810001,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359
109950740,2019-11-30 23:59:59 UTC,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.029999,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb
109950741,2019-11-30 23:59:59 UTC,view,2701706,2053013563911439225,appliances.kitchen.refrigerators,samsung,566.270020,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0


In [24]:
df_all.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109950743 entries, 0 to 109950742
Data columns (total 9 columns):
 #   Column         Non-Null Count      Dtype   
---  ------         --------------      -----   
 0   event_time     109950743 non-null  object  
 1   event_type     109950743 non-null  category
 2   product_id     109950743 non-null  uint32  
 3   category_id    109950743 non-null  uint64  
 4   category_code  74536963 non-null   object  
 5   brand          94609585 non-null   object  
 6   price          109950743 non-null  float32 
 7   user_id        109950743 non-null  uint32  
 8   user_session   109950731 non-null  object  
dtypes: category(1), float32(1), object(4), uint32(2), uint64(1)
memory usage: 5.4+ GB


## 데이터프레임 parquet로 저장하고 가져오기

In [25]:
file_path_parquet = 'data/2019-Oct-Nov.parquet.gzip'
# file_path_csv = 'data/2019-Oct-Nov.csv'

In [26]:
# 저장하기
start = time.time()
df_all.to_parquet(file_path_parquet, compression='gzip')
end = time.time()

print('걸린 시간 : ', (end - start), 'sec')

걸린 시간 :  251.28104281425476 sec


In [27]:
# 가져오기
start = time.time()
df_final = pd.read_parquet(file_path_parquet)
df_final.info()
end = time.time()

print('걸린 시간 : ', (end - start), 'sec')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109950743 entries, 0 to 109950742
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     uint32  
 3   category_id    uint64  
 4   category_code  object  
 5   brand          object  
 6   price          float32 
 7   user_id        uint32  
 8   user_session   object  
dtypes: category(1), float32(1), object(4), uint32(2), uint64(1)
memory usage: 5.4+ GB
걸린 시간 :  41.06379795074463 sec


## 데이터프레임 행 기준 8등분 저장하기

In [31]:
df_list = np.array_split(df_all, 8)
for n, sub_df in enumerate(df_list):
    sub_df.to_parquet(f"data/sub-df-{n}.parquet.gzip", compression='gzip')
end = time.time()

In [32]:
# 가져와서 확인해보기
pd.read_parquet('data/sub-df-1.parquet.gzip')

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
13743843,2019-10-11 13:50:07 UTC,view,12300305,2053013556311359947,construction.tools.drill,alteco,25.740000,548813932,4587b3ba-c9e0-41c2-9d5f-4c02b531e797
13743844,2019-10-11 13:50:07 UTC,view,28713027,2053013565639492569,apparel.shoes,baden,82.629997,559177601,4439348f-1166-4e68-aa15-736701b66512
13743845,2019-10-11 13:50:07 UTC,view,1004449,2053013555631882655,electronics.smartphone,xiaomi,200.750000,534264516,23b8c556-9a29-48e0-8452-c89bda432d25
13743846,2019-10-11 13:50:07 UTC,view,1004781,2053013555631882655,electronics.smartphone,huawei,271.549988,531547397,1ffe9738-169e-4c6d-a97b-3566a78114a4
13743847,2019-10-11 13:50:07 UTC,view,1005106,2053013555631882655,electronics.smartphone,apple,1541.839966,545696941,2ac4623b-7344-43a5-868e-9591e6fbf0e9
...,...,...,...,...,...,...,...,...,...
27487681,2019-10-20 16:33:43 UTC,view,3200353,2053013555321504139,appliances.kitchen.meat_grinder,vitek,66.849998,545462098,f00f06f0-14d1-49d6-9adc-64bf35f14595
27487682,2019-10-20 16:33:43 UTC,view,1004777,2053013555631882655,electronics.smartphone,xiaomi,135.009995,513451411,5ebfddd1-4162-49a4-9cb8-bce3a3b4edae
27487683,2019-10-20 16:33:43 UTC,view,2501672,2053013564003713919,appliances.kitchen.oven,bbk,85.690002,514098484,566b121b-96c4-4cc9-ba90-36cf9cf19d4e
27487684,2019-10-20 16:33:43 UTC,view,1201513,2172371436436455782,electronics.tablet,samsung,823.419983,514766113,db1e5b6f-9f9a-4928-8878-5e8141b33552
